In [1]:
from PIL import Image
from glob import glob
import os
from os import listdir
from os.path import isfile, join
import random
import shutil

import skimage
# from skimage.transform import rescale, resize, downscale_local_mean
import numpy
import cv2
# import glob
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import os

import scipy
from scipy import misc

import imageio
import argparse
import sys
# Will use matplotlib for showing the image
from matplotlib import pyplot as plt
print("TensorFlow version: " + tf.__version__)
print("SciPy version: " + scipy.__version__)

TensorFlow version: 1.14.0
SciPy version: 1.3.0


In [2]:
from __future__ import division, print_function, absolute_import

# Functions which need the PIL

import numpy
import tempfile

from numpy import (amin, amax, ravel, asarray, arange, ones, newaxis,
                   transpose, iscomplexobj, uint8, issubdtype, array)

try:
    from PIL import Image, ImageFilter
except ImportError:
    import Image
    import ImageFilter


if not hasattr(Image, 'frombytes'):
    Image.frombytes = Image.fromstring

In [3]:
photo_samples_from_each_category = 15 # max 100
sketch_samples_from_each_category = 15

In [4]:
root_folder = "/local/scratch/blacklleyt/Semester2/sketchypic/256x256"
photo_folder = root_folder + "/photo/tx_000100000000"
sketch_folder = root_folder + "/sketch/tx_000100000000"
destination = "/local/scratch/blacklleyt/Semester2/sketchypic/datasets"
salience_model = "/local/scratch/blacklleyt/Semester2/sketchypic/salience_model"
segment_script = "/local/scratch/blacklleyt/Semester2/sketchypic/segment/segment"

In [5]:
sigma_smoothing_value = 0.5
threhold_function_value = 500
minimum_component_size = 20

### Get the names of each of the categories in the sketchy dataset

In [6]:
category_names = []
for path in [f.path for f in os.scandir(photo_folder) if f.is_dir()]:
    category_names.append(os.path.basename(path))
print("category names: ", category_names)

category names:  ['chair', 'starfish', 'helicopter', 'cat', 'hermit_crab', 'banana', 'fish', 'hedgehog', 'airplane', 'guitar', 'frog', 'bell', 'lobster', 'seal', 'sailboat', 'duck', 'cup', 'pickup_truck', 'cabin', 'door', 'wine_bottle', 'candle', 'piano', 'camel', 'hat', 'violin', 'rocket', 'bicycle', 'pretzel', 'knife', 'raccoon', 'swan', 'pizza', 'hamburger', 'tiger', 'sheep', 'axe', 'teddy_bear', 'fan', 'ray', 'hot-air_balloon', 'racket', 'bear', 'volcano', 'tree', 'ant', 'scorpion', 'scissors', 'pineapple', 'owl', 'hammer', 'bread', 'teapot', 'table', 'alarm_clock', 'motorcycle', 'chicken', 'blimp', 'lion', 'crab', 'tank', 'jellyfish', 'strawberry', 'deer', 'rabbit', 'dog', 'squirrel', 'turtle', 'pear', 'apple', 'hotdog', 'penguin', 'armor', 'crocodilian', 'cow', 'car_(sedan)', 'ape', 'bat', 'hourglass', 'harp', 'castle', 'spider', 'pig', 'windmill', 'wheelchair', 'shoe', 'rifle', 'butterfly', 'wading_bird', 'bee', 'horse', 'sword', 'elephant', 'mushroom', 'trumpet', 'bench', 'kang

### Get the file paths of the photo and sketch files to load

In [7]:
photo_file_paths = []
sketch_file_paths = []
for category in category_names:
    photo_category_folder_path = photo_folder + "/" + category
    sketch_category_folder_path = sketch_folder + "/" + category
    all_photos = glob(photo_category_folder_path + "/*.jpg")
    all_sketches = glob(sketch_category_folder_path + "/*.png")
    random.Random(1).shuffle(all_photos)
    random.Random(1).shuffle(all_sketches)
    photo_file_paths = photo_file_paths + all_photos[:photo_samples_from_each_category]
    sketch_file_paths = sketch_file_paths + all_sketches[:sketch_samples_from_each_category]
    

### Copy the photos and sketches into collective folders

In [8]:
if not os.path.exists(destination):
    os.makedirs(destination)
    
print("length: " + str(len(photo_file_paths)))
    
photo_destination = destination + "/photo_original"
sketch_destination = destination + "/sketch"
    
if not os.path.exists(photo_destination):
    os.makedirs(photo_destination)
    
if not os.path.exists(sketch_destination):
    os.makedirs(sketch_destination)
    
for photo_path in photo_file_paths:
    filename = os.path.basename(photo_path)
    shutil.copy(photo_path, photo_destination) 
    
for sketch_path in sketch_file_paths:
    filename = os.path.basename(sketch_path)
    shutil.copy(sketch_path, sketch_destination)

length: 1875


### Generate salience maps for each of the photos

In [9]:
def fromimage(im, flatten=False, mode=None):
    if not Image.isImageType(im):
        raise TypeError("Input is not a PIL image.")

    if mode is not None:
        if mode != im.mode:
            im = im.convert(mode)
    elif im.mode == 'P':
        # Mode 'P' means there is an indexed "palette".  If we leave the mode
        # as 'P', then when we do `a = array(im)` below, `a` will be a 2-D
        # containing the indices into the palette, and not a 3-D array
        # containing the RGB or RGBA values.
        if 'transparency' in im.info:
            im = im.convert('RGBA')
        else:
            im = im.convert('RGB')

    if flatten:
        im = im.convert('F')
    elif im.mode == '1':
        # Workaround for crash in PIL. When im is 1-bit, the call array(im)
        # can cause a seg. fault, or generate garbage. See
        # https://github.com/scipy/scipy/issues/2138 and
        # https://github.com/python-pillow/Pillow/issues/350.
        #
        # This converts im from a 1-bit image to an 8-bit image.
        im = im.convert('L')

    a = array(im)
    return a


def bytescale(data, cmin=None, cmax=None, high=255, low=0):
    if data.dtype == uint8:
        return data

    if high > 255:
        raise ValueError("`high` should be less than or equal to 255.")
    if low < 0:
        raise ValueError("`low` should be greater than or equal to 0.")
    if high < low:
        raise ValueError("`high` should be greater than or equal to `low`.")

    if cmin is None:
        cmin = data.min()
    if cmax is None:
        cmax = data.max()

    cscale = cmax - cmin
    if cscale < 0:
        raise ValueError("`cmax` should be larger than `cmin`.")
    elif cscale == 0:
        cscale = 1

    scale = float(high - low) / cscale
    bytedata = (data - cmin) * scale + low
    return (bytedata.clip(low, high) + 0.5).astype(uint8)


def toimage(arr, high=255, low=0, cmin=None, cmax=None, pal=None,
            mode=None, channel_axis=None):
    
    data = asarray(arr)
    if iscomplexobj(data):
        raise ValueError("Cannot convert a complex-valued array.")
    shape = list(data.shape)
    valid = len(shape) == 2 or ((len(shape) == 3) and
                                ((3 in shape) or (4 in shape)))
    if not valid:
        raise ValueError("'arr' does not have a suitable array shape for "
                         "any mode.")
    if len(shape) == 2:
        shape = (shape[1], shape[0])  # columns show up first
        if mode == 'F':
            data32 = data.astype(numpy.float32)
            image = Image.frombytes(mode, shape, data32.tostring())
            return image
        if mode in [None, 'L', 'P']:
            bytedata = bytescale(data, high=high, low=low,
                                 cmin=cmin, cmax=cmax)
            image = Image.frombytes('L', shape, bytedata.tostring())
            if pal is not None:
                image.putpalette(asarray(pal, dtype=uint8).tostring())
                # Becomes a mode='P' automagically.
            elif mode == 'P':  # default gray-scale
                pal = (arange(0, 256, 1, dtype=uint8)[:, newaxis] *
                       ones((3,), dtype=uint8)[newaxis, :])
                image.putpalette(asarray(pal, dtype=uint8).tostring())
            return image
        if mode == '1':  # high input gives threshold for 1
            bytedata = (data > high)
            image = Image.frombytes('1', shape, bytedata.tostring())
            return image
        if cmin is None:
            cmin = amin(ravel(data))
        if cmax is None:
            cmax = amax(ravel(data))
        data = (data*1.0 - cmin)*(high - low)/(cmax - cmin) + low
        if mode == 'I':
            data32 = data.astype(numpy.uint32)
            image = Image.frombytes(mode, shape, data32.tostring())
        else:
            raise ValueError(_errstr)
        return image

    # if here then 3-d array with a 3 or a 4 in the shape length.
    # Check for 3 in datacube shape --- 'RGB' or 'YCbCr'
    if channel_axis is None:
        if (3 in shape):
            ca = numpy.flatnonzero(asarray(shape) == 3)[0]
        else:
            ca = numpy.flatnonzero(asarray(shape) == 4)
            if len(ca):
                ca = ca[0]
            else:
                raise ValueError("Could not find channel dimension.")
    else:
        ca = channel_axis

    numch = shape[ca]
    if numch not in [3, 4]:
        raise ValueError("Channel axis dimension is not valid.")

    bytedata = bytescale(data, high=high, low=low, cmin=cmin, cmax=cmax)
    if ca == 2:
        strdata = bytedata.tostring()
        shape = (shape[1], shape[0])
    elif ca == 1:
        strdata = transpose(bytedata, (0, 2, 1)).tostring()
        shape = (shape[2], shape[0])
    elif ca == 0:
        strdata = transpose(bytedata, (1, 2, 0)).tostring()
        shape = (shape[2], shape[1])
    if mode is None:
        if numch == 3:
            mode = 'RGB'
        else:
            mode = 'RGBA'

    if mode not in ['RGB', 'RGBA', 'YCbCr', 'CMYK']:
        raise ValueError(_errstr)

    if mode in ['RGB', 'YCbCr']:
        if numch != 3:
            raise ValueError("Invalid array shape for mode.")
    if mode in ['RGBA', 'CMYK']:
        if numch != 4:
            raise ValueError("Invalid array shape for mode.")

    # Here we know data and mode is correct
    image = Image.frombytes(mode, shape, strdata)
    return image

def imresize(arr, size, interp='bilinear', mode=None):
    im = toimage(arr, mode=mode)
    ts = type(size)
    if issubdtype(ts, numpy.signedinteger):
        percent = size / 100.0
        size = tuple((array(im.size)*percent).astype(int))
    elif issubdtype(type(size), numpy.floating):
        size = tuple((array(im.size)*size).astype(int))
    else:
        size = (size[1], size[0])
    func = {'nearest': 0, 'lanczos': 1, 'bilinear': 2, 'bicubic': 3, 'cubic': 3}
    imnew = im.resize(size, resample=func[interp])
    return fromimage(imnew)

In [10]:
photo_salience_map_destination = destination + "/photo_salience_map"
    
if not os.path.exists(photo_salience_map_destination):
    os.makedirs(photo_salience_map_destination)

g_mean = numpy.array(([126.88,120.24,112.19])).reshape([1,1,3])
gpu_fraction = 1.0
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = gpu_fraction)
with tf.Session(config=tf.ConfigProto(gpu_options = gpu_options)) as sess:
    saver = tf.train.import_meta_graph(salience_model + '/meta_graph/my-model.meta')
    saver.restore(sess,tf.train.latest_checkpoint(salience_model))
    image_batch = tf.get_collection('image_batch')[0]
    pred_mattes = tf.get_collection('mask')[0]
    
    for photo_path in photo_file_paths:
        filename = os.path.basename(photo_path)
        rgb = cv2.imread(photo_path)
        if rgb.shape[2]==4:
            rgb = rgba2rgb(rgb)
        origin_shape = rgb.shape
        rgb = numpy.expand_dims(imresize(rgb.astype(numpy.uint8),[320,320,3]).astype(numpy.float32)-g_mean,0)
        feed_dict = {image_batch:rgb}
        pred_alpha = sess.run(pred_mattes,feed_dict = feed_dict)
        final_alpha = imresize(numpy.squeeze(pred_alpha),origin_shape)
        cv2.imwrite((photo_salience_map_destination + "/" + filename),final_alpha)

W0830 13:54:10.096683 140501534275392 deprecation.py:323] From /usr/pkg/lib/python3.7/site-packages/tensorflow/python/training/queue_runner_impl.py:391: QueueRunner.__init__ (from tensorflow.python.training.queue_runner_impl) is deprecated and will be removed in a future version.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
W0830 13:54:10.106972 140501534275392 deprecation.py:323] From /usr/pkg/lib/python3.7/site-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [11]:
photo_salience_binary_mask_destination = destination + "/photo_salience_binary_mask"

if not os.path.exists(photo_salience_binary_mask_destination):
    os.makedirs(photo_salience_binary_mask_destination)

for photo_path in photo_file_paths:
    filename = os.path.basename(photo_path)

    image = cv2.imread(photo_salience_map_destination + "/" + filename, cv2.IMREAD_GRAYSCALE)

    (thresh, im_bw) = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    before = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
    after = cv2.cvtColor(im_bw,cv2.COLOR_GRAY2RGB)
    #plt.imshow(np.hstack((before,after)))

    cv2.imwrite(photo_salience_binary_mask_destination + "/" + filename, im_bw)

In [12]:
photo_salience_extracted = destination + "/photo_salience_extracted"

if not os.path.exists(photo_salience_extracted):
    os.makedirs(photo_salience_extracted)

for photo_path in photo_file_paths:
    filename = os.path.basename(photo_path)
    filename_without_extension = os.path.splitext(filename)[0]
    mask = cv2.imread(photo_salience_binary_mask_destination + "/" + filename, cv2.IMREAD_COLOR)
    original = cv2.imread(photo_path, cv2.IMREAD_COLOR)
    inverted_mask = cv2.bitwise_not(mask)
    subject = cv2.bitwise_and(original, mask)
    extracted = cv2.bitwise_or(subject, inverted_mask)
    original_rgb = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    original_and_mask = np.hstack((original_rgb, mask))
    extracted_rgb = cv2.cvtColor(extracted, cv2.COLOR_BGR2RGB)
    original_and_mask_and_extracted = np.hstack((original_and_mask, extracted_rgb))
    
#     cv2.imwrite(photo_salience_extracted + "/" + filename, extracted)
    final = Image.fromarray(cv2.cvtColor(extracted, cv2.COLOR_BGR2RGB))
    final.save(photo_salience_extracted + "/" + filename_without_extension + ".png")

### Convert the JPEG photo files to PPM as preparation for segment detection

In [13]:
if not os.path.exists(destination):
    os.makedirs(destination)
    
photo_ppm_destination = destination + "/photo_ppm"
    
if not os.path.exists(photo_ppm_destination):
    os.makedirs(photo_ppm_destination)
    
for photo_path in photo_file_paths:
    filename = os.path.basename(photo_path)
    filename_without_extension = os.path.splitext(filename)[0]
    photo = Image.open(photo_salience_extracted + "/" + filename_without_extension + ".png")
    photo.save(photo_ppm_destination + "/" + filename_without_extension + ".ppm")

### Perform segment detection on the photos

In [14]:
if not os.path.exists(destination):
    os.makedirs(destination)
    
photo_segment_ppm_destination = destination + "/photo_segments_ppm"
    
if not os.path.exists(photo_segment_ppm_destination):
    os.makedirs(photo_segment_ppm_destination)
    
ppm_photos = glob(photo_ppm_destination + "/*.ppm")

for photo_path in ppm_photos:
    filename = os.path.basename(photo_path)
    output_file = photo_segment_ppm_destination + "/" + filename
    bash_command = "/local/scratch/blacklleyt/Semester2/sketchypic/segment/segment "
    bash_command = bash_command + str(sigma_smoothing_value) + " "
    bash_command = bash_command + str(threhold_function_value) + " "
    bash_command = bash_command + str(minimum_component_size) + " "
    bash_command = bash_command + photo_path + " " + output_file
    os.system(bash_command)

### Convert the PPM segment files into PNG image files

In [15]:
photo_segment_destination = destination + "/photo_segments"
    
if not os.path.exists(photo_segment_destination):
    os.makedirs(photo_segment_destination)
    
photo_segment_ppm_photos = glob(photo_segment_ppm_destination + "/*.ppm")
    
for photo_path in photo_segment_ppm_photos:
    filename = os.path.basename(photo_path)
    filename_without_extension = os.path.splitext(filename)[0]
    photo = Image.open(photo_path)
    photo.save(photo_segment_destination + "/" + filename_without_extension + ".png")

In [16]:
# photo_segment_blurred_destination = destination + "/photo_segment_blurred_destination"
    
# if not os.path.exists(photo_segment_blurred_destination):
#     os.makedirs(photo_segment_blurred_destination)
    
# photo_segment_photos = glob(photo_segment_destination + "/*.png")
    
# for photo_path in photo_segment_photos:
#     filename = os.path.basename(photo_path)
#     filename_without_extension = os.path.splitext(filename)[0]
#     photo = Image.open(photo_path)
#     blurred = photo.filter(ImageFilter.GaussianBlur(radius=1))
#     blurred.save(photo_segment_blurred_destination + "/" + filename_without_extension + ".png")

### Apply Sobel operator to the detected segment images.

In [17]:
photo_segment_sobel_destination = destination + "/photo_segment_sobel_destination"
    
if not os.path.exists(photo_segment_sobel_destination):
    os.makedirs(photo_segment_sobel_destination)
    
photo_segment_photos = glob(photo_segment_destination + "/*.png")
    
for photo_path in photo_segment_photos:
    filename = os.path.basename(photo_path)
    filename_without_extension = os.path.splitext(filename)[0]
    image = cv2.imread(photo_path)
    image = image.astype('int32')
    dx = scipy.ndimage.sobel(image, 0)  # horizontal derivative
    dy = scipy.ndimage.sobel(image, 1)  # vertical derivative
    mag = numpy.hypot(dx, dy)  # magnitude
    mag *= 255.0 / numpy.max(mag)  # normalize (Q&D)
    cv2.imwrite(photo_segment_sobel_destination + "/" + filename, mag)

### Invert the output from from the Sobel filter.

In [18]:
if not os.path.exists(destination):
    os.makedirs(destination)

photo_segment_sobel_inverted_destination = destination + "/photo_segment_sobel_inverted_destination"
    
if not os.path.exists(photo_segment_sobel_inverted_destination):
    os.makedirs(photo_segment_sobel_inverted_destination)
    
photo_segment_sobel_photos = glob(photo_segment_sobel_destination + "/*.png")
    
for photo_path in photo_segment_sobel_photos:
    filename = os.path.basename(photo_path)
    filename_without_extension = os.path.splitext(filename)[0]
    image = cv2.imread(photo_path)
    image = cv2.bitwise_not(image)
    cv2.imwrite(photo_segment_sobel_inverted_destination + "/" + filename, image)

### Split into training and test datasets

In [19]:
train_folder = destination + "/train"
test_folder = destination + "/test"

photo_train_folder = train_folder + "/photo"
photo_test_folder = test_folder + "/photo"

sketch_train_folder = train_folder + "/sketch"

if not os.path.exists(train_folder):
    os.makedirs(train_folder)
    
if not os.path.exists(test_folder):
    os.makedirs(test_folder)
    
if not os.path.exists(photo_train_folder):
    os.makedirs(photo_train_folder)
    
if not os.path.exists(photo_test_folder):
    os.makedirs(photo_test_folder)
    
if not os.path.exists(sketch_train_folder):
    os.makedirs(sketch_train_folder)

# all_photos = glob(photo_salience_extracted + "/*.png")
all_photos = glob(photo_segment_sobel_inverted_destination + "/*.png")
all_sketches = glob(sketch_destination + "/*.png")

random.Random(2).shuffle(all_photos)

split = int(round(0.04 * len(all_photos)))

train_photos = all_photos[split:]
test_photos = all_photos[:split]

for photo_path in train_photos:
    filename = os.path.basename(photo_path)
    shutil.copy(photo_path, photo_train_folder) 
    
for photo_path in test_photos:
    filename = os.path.basename(photo_path)
    shutil.copy(photo_path, photo_test_folder) 
    
for sketch_path in all_sketches:
    filename = os.path.basename(sketch_path)
    shutil.copy(sketch_path, sketch_train_folder) 